# Synth. Obs.: 3D Phantom - Reduction - NLTE

We create synthetic observations for the Magritte model of the 3D Phantom snapshot that was created in the [this example](../1_post-processing/3_create_Phantom_3D.ipynb), after computing the radiation field self-consistently.

## Setup

Import the required functionalty.

In [1]:
from magrittetorch.model.model import Model
import magrittetorch.tools.plot as plot
import magrittetorch.algorithms.solvers as solvers
from astropy import units, constants
import os
import torch

Define a working directory (you will have to change this). We assume here that the scripts of the [this example](../1_post-processing/3_create_Phantom_3D.ipynb) have already been executed and go back to that working directory.

In [2]:
wdir = "/lhome/thomasc/Magrittetorch-examples/Phantom_3D/"

Define file names.

In [3]:
model_file = os.path.join(wdir, 'model_Phantom_3D_red.hdf5')   # 3D Phantom Magrittetorch model

Load the Magritte model.

In [4]:
model = Model(model_file)
model.read()

Reading model from:  /lhome/thomasc/Magrittetorch-examples/Phantom_3D/model_Phantom_3D_red.hdf5
Reading Magrittetorch model


## Model the medium

Initialize the model

In [5]:
model.dataCollection.infer_data()

In this example we will work with the LTE level populations and **will demand** statistical equilibrium.

In [6]:
# If you have a GPU, we can use it to speed up the computations
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Getting the full model on GPU might be a bit too much for its memory, so you might want to use your CPU instead for the computations
# device = torch.device("cpu")

# Compute level populations
solvers.compute_level_populations(model, device=device, max_n_iterations=20)

it: 0
rr: 0
rr: 1
rr: 2
rr: 3
rr: 4
rr: 5
rr: 6
rr: 7
rr: 8
rr: 9
rr: 10
rr: 11
rr: 12
rr: 13
rr: 14
rr: 15
rr: 16
rr: 17
rr: 18
rr: 19
rr: 20
rr: 21
rr: 22
rr: 23
rr: 24
rr: 25
rr: 26
rr: 27
rr: 28
rr: 29
rr: 30
rr: 31
rr: 32
rr: 33
rr: 34
rr: 35
rr: 36
rr: 37
rr: 38
rr: 39
rr: 40
rr: 41
rr: 42
rr: 43
rr: 44
rr: 45
rr: 46
rr: 47
time for this iteration:  49.78686785697937 s
total time elapsed: 49.970887184143066 s
Fraction converged: 6.753707016165432e-05
Model converged: False
it: 1
rr: 0
rr: 1
rr: 2
rr: 3
rr: 4
rr: 5
rr: 6
rr: 7
rr: 8
rr: 9
rr: 10
rr: 11
rr: 12
rr: 13
rr: 14
rr: 15
rr: 16
rr: 17
rr: 18
rr: 19
rr: 20
rr: 21
rr: 22
rr: 23
rr: 24
rr: 25
rr: 26
rr: 27
rr: 28
rr: 29
rr: 30
rr: 31
rr: 32
rr: 33
rr: 34
rr: 35
rr: 36
rr: 37
rr: 38
rr: 39
rr: 40
rr: 41
rr: 42
rr: 43
rr: 44
rr: 45
rr: 46
rr: 47
time for this iteration:  50.09438753128052 s
total time elapsed: 111.38907432556152 s
Fraction converged: 0.5762516926060282
Model converged: False
it: 2
rr: 0
rr: 1
rr: 2
rr: 3
rr: 4

## Make synthetic observations

Now we can make synthetic observations of the model.

In [7]:
# Get the line frequency corresponding to the CO 2-1 line
fcen = model.sources.lines.lineProducingSpecies[0].linedata.frequency.get(device)[1]
vpix = 1500   # velocity pixel size [m/s] 
nfreqs = 31   # number of frequency bins
dd   = vpix * (nfreqs-1)/2 / constants.c.value # frequency bin size [Hz]
fmin = fcen - fcen*dd
fmax = fcen + fcen*dd

image_freqs = torch.linspace(fmin, fmax, nfreqs, device=device, dtype=torch.float32)

# Compute image in the specified direction at the given frequencies
solvers.image_model(model, torch.Tensor([0,0,1]).type(torch.float32).to(device), image_freqs, device)

## Plot observations

Plot the resulting channel maps with the Magritte matplotlib back end.

In [8]:
plot.image_mpl(
    model,
    image_nr =  -1,
    zoom     = 1.3,
    npix_x   = 256,
    npix_y   = 256,
    x_unit   = units.au,
    v_unit   = units.km / units.s,
    method = 'nearest'
)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 31/31 [00:15<00:00,  2.02it/s]


interactive(children=(IntSlider(value=15, description='v', max=30), Output()), _dom_classes=('widget-interact'…

<function magrittetorch.tools.plot.image_mpl.<locals>.<lambda>(v)>

(The plot is only interactive in a live notebook.)

Save the image cube in a fits file.

In [9]:
plot.save_fits(model)

Written file to: /lhome/thomasc/Magrittetorch-examples/Phantom_3D/images/image.fits


(Optional: To create your own plots) Overview of data stored in the Image object

In [10]:
import numpy as np
xdir = np.array(model.images[-1].image_direction_x.get())#directions of the x-and y-vectors of the image
ydir = np.array(model.images[-1].image_direction_y.get())
zdir = np.array(model.images[-1].image_direction_z.get())#this is direction in which we observe the object
print("image directions: ", xdir, ydir, zdir)
nfreqs = np.array(model.images[-1].nfreqs.get()) #number of frequency bins
freqs = np.array(model.images[-1].freqs.get_astropy()) #frequency bins [Hz]
print("# of frequencies: ", nfreqs, " frequencies :", freqs)
ImX = np.array(model.images[-1].imX.get_astropy())#X position in image [m]
ImY = np.array(model.images[-1].imY.get_astropy())#Y position in image [m]
I = np.array(model.images[-1].I.get_astropy())#Intensity at the corresponding ImX, ImY position [W/(m^2*Hz*sr)], at a given frequency bin
# print("Intensities :", I, " ImX:", ImX, "ImY:", ImY) #prints a lot of output

image directions:  [1. 0. 0.] [0.000000e+00 1.000000e+00 4.371139e-08] [ 0.000000e+00 -4.371139e-08  1.000000e+00]
# of frequencies:  31  frequencies : [2.3051970e+11 2.3052085e+11 2.3052201e+11 2.3052316e+11 2.3052432e+11
 2.3052547e+11 2.3052662e+11 2.3052778e+11 2.3052893e+11 2.3053009e+11
 2.3053124e+11 2.3053238e+11 2.3053355e+11 2.3053469e+11 2.3053586e+11
 2.3053700e+11 2.3053815e+11 2.3053931e+11 2.3054046e+11 2.3054162e+11
 2.3054277e+11 2.3054392e+11 2.3054508e+11 2.3054623e+11 2.3054739e+11
 2.3054854e+11 2.3054968e+11 2.3055085e+11 2.3055199e+11 2.3055316e+11
 2.3055430e+11]
